## 今回の目的
与えられた10分間の取引におけるボラティリティから次の10分のボラティリティを予測する

→stock_id（銘柄）ごとのtime_id（時間）ごとのボラティリティを予測する

→ボラティリティについての詳細はのちに解説

================================================================================

⚠️kaggle初心者が強引に和訳しながら流れを確認したので解釈が間違っている可能性があります。ご注意ください。

================================================================================

## データの概要
### train.csv
stock_id, time_id, target（ボラティリティ）が含まれる
### test.csv
stock_idが0のものの、time_idが4/32/34の3行のみ
→row_idという列に0-4などstock_idとtime_idをハイフンで接続したものが含まれている
### sample_submission.csv
row_id, targetの3行2列のみ（row_id = 0-4, 0-32, 0-34）
### book_train.parquet/book_test.parquet
stock_idごとのオーダーブックのデータがparquetという形で入っている（testはstock_id=0のみ）
### trade_train.parquet/trade_test.parquet
stock_idごとの実際のトレードデータがparquetという形で入っている（testはstock_id=0のみ）

In [ ]:
# book_train.parquetのデータを確認
import pandas as pd
book_example = pd.read_parquet('../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=0')
book_example.head()

### オーダーブックのイメージ（公式より）
![](https://i.imgur.com/16Qt255l.png)
- bid_price1:最高売値。画像でいうと147にあたる
- ask_price1:最低買値。画像でいうと148にあたる
- bid_price2:2番目に高い売値。画像でいうと146にあたる
- ask_price2:2番目に低い買値。画像でいうと149にあたる
- bid_size1:bid_price1の時の数。画像でいうと251
- ask_size1:ask_price1の時の数。画像でいうと221
- bid_size2:bid_price2の時の数。画像でいうと321
- ask_size2:ask_price2の時の数。画像でいうと148

seconds_in_bucketは必ず0から始まり、オーダーブックに何かしらの動きがあると随時レコードが追加される。
seconds_in_bucketが1の時はask_size1が226から100に変化したため、レコードが追加された。

In [ ]:
# trade_train.parquetのデータを確認
import pandas as pd
trade_example = pd.read_parquet('../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=0')
trade_example.head()

オーダーブックをもとに実際に売買が行われるとtradeデータにレコードが追加される。
最初のレコードでは、
time_id=5の21秒の時にprice=1.002301で株式数326、取引注文12のやりとりが行われた

## ボラティリティとは？


ボラティリティとは何かを理解する前にWAPについて意味合いを理解する
### WAP(Weighted averaged price)
日本語訳では加重平均。
ウェイトを考慮した平均値のようなイメージで下記の式で計算される
$$
    𝑊𝐴𝑃 = \frac{𝐵𝑖𝑑𝑃𝑟𝑖𝑐𝑒1∗𝐴𝑠𝑘𝑆𝑖𝑧𝑒1+𝐴𝑠𝑘𝑃𝑟𝑖𝑐𝑒1∗𝐵𝑖𝑑𝑆𝑖𝑧𝑒1}{𝐵𝑖𝑑𝑆𝑖𝑧𝑒1+𝐴𝑠𝑘𝑆𝑖𝑧𝑒1}
$$
𝐴𝑠𝑘𝑆𝑖𝑧𝑒1と𝐵𝑖𝑑𝑆𝑖𝑧𝑒1に注目した際に、𝐴𝑠𝑘𝑆𝑖𝑧𝑒が大きくなれば𝐵𝑖𝑑𝑃𝑟𝑖𝑐𝑒を掛ける割合も全体に対して大きくなり、計算された値は𝐵𝑖𝑑𝑃𝑟𝑖𝑐𝑒と𝐴𝑠𝑘𝑃𝑟𝑖𝑐𝑒では𝐵𝑖𝑑𝑃𝑟𝑖𝑐𝑒に近づく

このようにそれぞれを平等に考えるのではなく、要素の大きさを考慮した上で平均値を取得する考え方がWAP

### Log retuen
株価の割合の比率を対数で取得した値。logの割り算は引き算に変換できるため、ある時間t2でのWAPからある時間t1でのWAPのそれぞれの対数を引いた値
$$
𝑟𝑡1,𝑡2 = \log(\frac{WAP t2}{WAPt1})
$$
$$
= \log(WAP t2) - \log(WAP t1)
$$

## Realized volatility
上で取得したLog returnの標準偏差を１年間分計算した値である年間標準偏差をボラティリティとよぶ。
$$
𝜎 = \sqrt{\sum_{t}^{}𝑟^2𝑡-1,𝑡\quad}
$$
今回は10分（second_in_bucketが0から600）単位でLog returnを求めことを一年分行い、標準偏差を求める。

ボラティリティが高いほど売買の動きが盛んであるということになり、手数料で会社が儲かるからいい市場ということ（？）

## lgbm baselineを日本語で解釈
most votes上位であった「lgbm baseline」の分析を踏襲しつつ、コンペの理解を深めていく。
下記から実際のコード（コメントアウト分は適宜追加）

In [ ]:
# 好きなHTMLを読み込めるようになるやつ（画像挿入などに使うのか？）
from IPython.core.display import display, HTML

import pandas as pd
import numpy as np # linear algebra
# なぜか2回importしてたので無視
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# 引数に指定されたパターンにマッチするファイルパス名を取得してくれるモジュール
# 今回は使っていなかった
import glob

# osに依存している機能を利用するためのモジュール
# 例えばos.walk()でファイルやディレクトリの一覧を取得できる
# os.system("ls")でUnixコマンド同様の結果が得られる
# 今回は使っていなかった
import os

# ガベージコレクション（プログラムは実行中に必要なメモリ領域を動的に確保するが、不要になったメモリ領域を自動的に解放する機能。）
import gc

# 並列化
# multiproccessingというモジュールも他にあるっぽいが今回はjoblib
# CPUを複数使ってくれるようになるらしい。これがあるのと無いのとでは処理の時間が全然違う
from joblib import Parallel, delayed

from sklearn import preprocessing, model_selection
from sklearn.preprocessing import MinMaxScaler #正規化（最小0,最大1となるように変換）
from sklearn.preprocessing import QuantileTransformer #分位による変換
from sklearn.metrics import r2_score #決定係数を計算する用のモジュール

import matplotlib.pyplot as plt
import seaborn as sns
import numpy.matlib

# matplotlibはできることが多くていいけど、扱うのが複雑で面倒
# seabornはmatplotlibをベースにしているけど、もっと見やすくて綺麗で、簡単にかけるから最高らしい

# 設定したものの今回は使っていなかった
path_submissions = '/'

# submissionのカラム名に合わせて
target_name = 'target'
# 空の辞書型を用意（おそらく提出用）
scores_folds = {}

## WAPを計算する関数を作成

In [ ]:
# data directory
# Kaggleをする際の環境を区別してPATHを設定するコード
# 今回はbookとtradeのデータがディレクトリが大量にあってその中にそれぞれのファイルが格納されているためpathを最初に指定してしまうのが楽
data_dir = '../input/optiver-realized-volatility-prediction/'

# 1番目のWAP(加重平均)を計算する関数（最高売値とその個数、最低買値とその個数を使って計算）
# 引数にdfを設定
def calc_wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

# 2番目のWAP(加重平均)を計算する関数（2番目に高い売値とその個数、2番目に低い買値とその個数を使って計算）
def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

# WAPとしているがbid同士ask同士を掛け合わせているため若干意味合いが違う
def calc_wap3(df):
    wap = (df['bid_price1'] * df['bid_size1'] + df['ask_price1'] * df['ask_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

# wap3の2番目ver
def calc_wap4(df):
    wap = (df['bid_price2'] * df['bid_size2'] + df['ask_price2'] * df['ask_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

WAPとは別でbid同士ask同士を掛け合わせたものをなぜ作成したのかは不明

## Log returnを計算する関数を作成

In [ ]:
# Log returnの関数を作成
# np.log(series).diff() = np.log(series) - np.log(series.shift()) （一個前の対数の差分）
def log_return(series):
    return np.log(series).diff()

## 目的変数ボラティリティを計算する関数

In [ ]:
# ボラティリティを計算するための関数
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

In [ ]:
# 重複なく数えた時の個数を計算するための関数
# np.unique(series)でseriesのユニークな値が一覧で取得され、それのlen()なので個数が取得される
def count_unique(series):
    return len(np.unique(series))

前置きで解説したWAP, Log return, ボラティリティを計算するための関数を設定

続いてデータの読み込み

## 訓練データとテストデータの読み込み

In [ ]:
# 訓練データとテストデータを読み込むための関数
def read_train_test():
    train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
    test = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv')
    # bookデータとtradeデータを結合するためのidとなるカラムを訓練データとテストデータにそれぞれに追加
    train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
    test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)
    print(f'Our training set has {train.shape[0]} rows')
    return train, test

In [ ]:
# trainとtestの概要をチェック
read_train_test()

それぞれにrow_idが追加されたためmergeが可能

## オーダーブックとトレードデータを集計した上で結合するための関数

In [ ]:
# オーダーブックのデータを加工していく
# それぞれの銘柄（stock_id）ごとに前処理をするための関数を設定
def book_preprocessor(file_path):
    # ApacheParquetとはcsvなどの行志向のデータフォーマットと違い、列志向のフォーマットで、列単位でデータを取り出す分析用途に向いてる
    df = pd.read_parquet(file_path)
    # WAP(加重平均)を計算した上で特徴量として追加
    df['wap1'] = calc_wap1(df)
    df['wap2'] = calc_wap2(df)
    df['wap3'] = calc_wap3(df)
    df['wap4'] = calc_wap4(df)
    # Log retuenを計算した上で特徴量として追加
    # applyでdataframeに対して全ての操作を行う
    # 今回はtime_idごとのそれぞれのwapの列に対してlog_returnを行う
    df['log_return1'] = df.groupby(['time_id'])['wap1'].apply(log_return)
    df['log_return2'] = df.groupby(['time_id'])['wap2'].apply(log_return)
    df['log_return3'] = df.groupby(['time_id'])['wap3'].apply(log_return)
    df['log_return4'] = df.groupby(['time_id'])['wap4'].apply(log_return)
    # WAP1とWAP2同士の差分の絶対値を取得し特徴量として追加
    df['wap_balance'] = abs(df['wap1'] - df['wap2'])
    # spread（最高売値と最低買値の比率をとって計算される値）を計算し、特徴量として追加
    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1']) / 2)
    df['price_spread2'] = (df['ask_price2'] - df['bid_price2']) / ((df['ask_price2'] + df['bid_price2']) / 2)
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df["bid_ask_spread"] = abs(df['bid_spread'] - df['ask_spread'])
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))
    
    # 辞書型を用意する
    # カラム名と行う操作をセット
    create_feature_dict = {
        'wap1': [np.sum, np.std],
        'wap2': [np.sum, np.std],
        'wap3': [np.sum, np.std],
        'wap4': [np.sum, np.std],
        'log_return1': [realized_volatility],
        'log_return2': [realized_volatility],
        'log_return3': [realized_volatility],
        'log_return4': [realized_volatility],
        'wap_balance': [np.sum, np.max],
        'price_spread':[np.sum, np.max],
        'price_spread2':[np.sum, np.max],
        'bid_spread':[np.sum, np.max],
        'ask_spread':[np.sum, np.max],
        'total_volume':[np.sum, np.max],
        'volume_imbalance':[np.sum, np.max],
        "bid_ask_spread":[np.sum,  np.max],
    }
    create_feature_dict_time = {
        'log_return1': [realized_volatility],
        'log_return2': [realized_volatility],
        'log_return3': [realized_volatility],
        'log_return4': [realized_volatility],
    }
    
    # DataFrameの組み直し
    ## 引数のadd_suffixは下記（*）
    def get_stats_window(fe_dict,seconds_in_bucket, add_suffix = False):
        # dfの中でもdfの'seconds_in_bucket'カラムに入っている値がseconds_in_bucketより大きい時
        # 辞書型と作ってaggとreset_index()で新たにDataFrameを作るセットのような役割？
        # 引数に何も指定せずreset_index()を使うと、連番が新たなindexとなり、元のindexが新たな列として残る
        # これをやることで辞書型で設定した最大値や平均値が計算されたもののDataFrameとなるため欠損値が無視される
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(fe_dict).reset_index()
        # 接尾辞を結合する列の名前を変更する
        # wap1 sum → wap1_sumにしたい
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        
        # *
        # add_suffix：カラムのケツに「_秒数」をつける
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    # seconds_in_bucketを100秒ごとにcreate_feature_dictを計算してDataFrameとする
    df_feature = get_stats_window(create_feature_dict,seconds_in_bucket = 0, add_suffix = False)
    df_feature_500 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 500, add_suffix = True)
    df_feature_400 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 200, add_suffix = True)
    df_feature_100 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 100, add_suffix = True)

    # DataFrameの結合
    df_feature = df_feature.merge(df_feature_500, how = 'left', left_on = 'time_id_', right_on = 'time_id__500')
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')
    df_feature = df_feature.merge(df_feature_100, how = 'left', left_on = 'time_id_', right_on = 'time_id__100')
    # 不要になった結合用のカラムを削除
    df_feature.drop(['time_id__500','time_id__400', 'time_id__300', 'time_id__200','time_id__100'], axis = 1, inplace = True)
    
    
    # のちにtradeデータと結合する用のrow_idというカラムを作成
    # stock_id=0となっているディレクトリの0の部分だけを取り出し、stock_idに代入する
    stock_id = file_path.split('=')[1]
    # time_idとstock_idが繋がった「0-32」のような形のrow_idを作成
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x: f'{stock_id}-{x}')
    df_feature.drop(['time_id_'], axis = 1, inplace = True)
    return df_feature

In [ ]:
# トレードデータ

# オーダーブックと同じような操作を行う
def trade_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    # dfをtime_idごとにグループ化してそのうちの「price」カラムにlog_returnを適用する
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return)
    #  取引された合計金額を取得し特徴量として追加
    df['amount']=df['price']*df['size']
    # 辞書型を用意
    create_feature_dict = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum, np.max, np.min],
        'order_count':[np.sum,np.max],
        'amount':[np.sum,np.max,np.min],
    }
    create_feature_dict_time = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum],
        'order_count':[np.sum],
    }
    
    def get_stats_window(fe_dict,seconds_in_bucket, add_suffix = False):
        # bookと同じ処理
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(fe_dict).reset_index()
        # ケツに「_df_featureのカラム名」をつける
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        # ケツに「_秒数」とつけるための自作関数 
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    

    df_feature = get_stats_window(create_feature_dict,seconds_in_bucket = 0, add_suffix = False)
    df_feature_500 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 500, add_suffix = True)
    df_feature_400 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 200, add_suffix = True)
    df_feature_100 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 100, add_suffix = True)
    
    def tendency(price, vol):
        # priceの差分
        df_diff = np.diff(price)
        # 増加率的な
        val = (df_diff/price[1:])*100
        # 指定したボリュームに増加率をかけたものを合計した値
        power = np.sum(val*vol[1:])
        return(power)
    
    lis = []
    # n_tim_idはdf内のtime_idがユニークなもの
    for n_time_id in df['time_id'].unique():
        # ユニークなtime_idごとにdfのidを取得（time_idが0の時のdfをdf_id=0とする）
        df_id = df[df['time_id'] == n_time_id]
        # それぞれのdf_idごと（つまりtime_idごと）の値段と株式数をかけた合計取引金額
        tendencyV = tendency(df_id['price'].values, df_id['size'].values)
        # 平均金額よりも大きい金額かどうか
        f_max = np.sum(df_id['price'].values > np.mean(df_id['price'].values))
        f_min = np.sum(df_id['price'].values < np.mean(df_id['price'].values))
        df_max =  np.sum(np.diff(df_id['price'].values) > 0)
        df_min =  np.sum(np.diff(df_id['price'].values) < 0)
        # 標準偏差の絶対値の中央値？的なイメージ
        abs_diff = np.median(np.abs( df_id['price'].values - np.mean(df_id['price'].values)))
        # それぞれのpriceの2乗の平均
        energy = np.mean(df_id['price'].values**2)
        # 四分位点の引き算
        iqr_p = np.percentile(df_id['price'].values,75) - np.percentile(df_id['price'].values,25)
        
        # 株式数でも同じ操作を行う
        
        abs_diff_v = np.median(np.abs( df_id['size'].values - np.mean(df_id['size'].values)))        
        energy_v = np.sum(df_id['size'].values**2)
        iqr_p_v = np.percentile(df_id['size'].values,75) - np.percentile(df_id['size'].values,25)
        
        # 先ほど作った空の配列に用意したDataFrameを追加
        lis.append({'time_id':n_time_id,'tendency':tendencyV,'f_max':f_max,'f_min':f_min,'df_max':df_max,'df_min':df_min,
                   'abs_diff':abs_diff,'energy':energy,'iqr_p':iqr_p,'abs_diff_v':abs_diff_v,'energy_v':energy_v,'iqr_p_v':iqr_p_v})
    
    # DataFrame化
    df_lr = pd.DataFrame(lis)
        
   # df_featureに結合
    df_feature = df_feature.merge(df_lr, how = 'left', left_on = 'time_id_', right_on = 'time_id')
    
    # 100秒ごとに切ったそれぞれのdfも結合
    df_feature = df_feature.merge(df_feature_500, how = 'left', left_on = 'time_id_', right_on = 'time_id__500')
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')
    df_feature = df_feature.merge(df_feature_100, how = 'left', left_on = 'time_id_', right_on = 'time_id__100')
    # merge用のカラムを削除
    df_feature.drop(['time_id__500','time_id__400', 'time_id__300', 'time_id__200','time_id','time_id__100'], axis = 1, inplace = True)
    
    # add_suffixの先頭につけるver
    df_feature = df_feature.add_prefix('trade_')
    # こちらもrow_idを取得
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature.drop(['trade_time_id_'], axis = 1, inplace = True)
    return df_feature

オーダーブックとトレードデータを銘柄ごとに操作するための関数は設定完了

道中で行なっていた操作は学習に必要な特徴量を追加するためのもので、ここを後々自分でもどんなものが良いか考える必要がある

## stock_idとtime_idごとの集計を行うための関数を設定

stock_id=0だけでもデータ数が120万くらいあるため総数で1億は超えそう。集計しておかないとpcが落ちる

In [ ]:
# stock_idとtime_idごとの集計を行うための関数を設定
def get_time_stock(df):
    # カラム名を一次元配列で指定
    vol_cols = ['log_return1_realized_volatility', 'log_return2_realized_volatility', 'log_return1_realized_volatility_400', 'log_return2_realized_volatility_400', 
                'log_return1_realized_volatility_300', 'log_return2_realized_volatility_300', 'log_return1_realized_volatility_200', 'log_return2_realized_volatility_200', 
                'trade_log_return_realized_volatility', 'trade_log_return_realized_volatility_400', 'trade_log_return_realized_volatility_300', 'trade_log_return_realized_volatility_200']

    # stock_idごとに集計（平均、標準偏差、最大値、最小値を取得）
    df_stock_id = df.groupby(['stock_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    # stockの方だとわかるようにケツに「_stock」と追加
    df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
    df_stock_id = df_stock_id.add_suffix('_' + 'stock')

    # time_idごとに集計（平均、標準偏差、最大値、最小値を取得）
    df_time_id = df.groupby(['time_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    # timeの方だとわかるようにケツに「_time」と追加
    df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
    df_time_id = df_time_id.add_suffix('_' + 'time')
    
    # dfにstock_idごとに集計したものとtime_idごとに集計したものを結合
    df = df.merge(df_stock_id, how = 'left', left_on = ['stock_id'], right_on = ['stock_id__stock'])
    df = df.merge(df_time_id, how = 'left', left_on = ['time_id'], right_on = ['time_id__time'])
    df.drop(['stock_id__stock', 'time_id__time'], axis = 1, inplace = True)
    return df

## 並列処理

In [ ]:
# 並列化
# 今までに作った前処理用の関数を並列化するための関数
def preprocessor(list_stock_ids, is_train = True):
    
    # Parallel for loop
    def for_joblib(stock_id):
        if is_train:
            # is_train=Trueと指定された場合trainデータの方へのpathを作成
            file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
        else:
            # is_train=Falseつまりtestデータならこちらへのpathを作成
            file_path_book = data_dir + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_test.parquet/stock_id=" + str(stock_id)
    
        # bookとtradeを前処理した上で結合する
        # 些細な変化も全てレコードとして記録されているbookの方にrow_idを基準にtradeを外部結合する
        df_tmp = pd.merge(book_preprocessor(file_path_book), trade_preprocessor(file_path_trade), on = 'row_id', how = 'left')
        
        # 結合したDataFrameを返す
        return df_tmp
    
    # 並列APIを使用してfor_joblib関数を呼び出す
    # n_jobs:最大同時実行ジョブ数。-1とすると全てのCPUが使用される
    # verbose:ログの出力レベル（冗長性）。デフォルトでは何も出力されない。値を大きくすると出力レベルが上がる（冗長性が増す）。10より大きいとすべてのログが出力され、50以上だとstdout（標準出力）に出力される。
    # delayed(<実行する関数>)(<関数への引数>) for 変数名 in イテラブル
    # 実行する関数（bookとtradeを前処理して結合）をstock_idごとに行う
    df = Parallel(n_jobs = -1, verbose = 1)(delayed(for_joblib)(stock_id) for stock_id in list_stock_ids)
    # Parallelから返されるすべてのDataFrameを結合
    # ignore_index=Trueでindexがconcat前のindexを無視して連番で振られる
    df = pd.concat(df, ignore_index = True)
    return df

## RMSPEで無駄な処理が走るのを止める

In [ ]:
# RMSPE(平均平方二乗誤差率)
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

In [ ]:
# RMSPEで早期停止する関数
def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

## 訓練データとtestデータを作成した関数によって読み込む

In [ ]:
# trainデータとtestデータの読み込み
train, test = read_train_test()

# ユニークなstockのidを取得する
train_stock_ids = train['stock_id'].unique()
# 並列処理に使うために前処理を行う
# 今回は並列化してbookとtradeをstock_idごとに処理をした上で結合する
train_ = preprocessor(train_stock_ids, is_train = True)
# row_idを基準にleft joinする
train = train.merge(train_, on = ['row_id'], how = 'left')

# テストデータに対しても同じ処理を行う
test_stock_ids = test['stock_id'].unique()
test_ = preprocessor(test_stock_ids, is_train = False)
test = test.merge(test_, on = ['row_id'], how = 'left')

# time_idとstock_idが一緒になったデータを取得
# get_time_stockはstock_idとtime_idごとの集計を行う関数
train = get_time_stock(train)
test = get_time_stock(test)

kaggleのnotebookだと30~40分くらいかかる。データ数が1億超えてるからしょうがない、、

## 時定数tauを定義

In [ ]:
# 時定数tau
# ニューラルネットワークで必要？
# seconds_in_bucket_countがあって接頭語に「trade_」と「_unique」をつけたもの
train['size_tau'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique'] )
test['size_tau'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique'] )
#train['size_tau_450'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_450'] )
#test['size_tau_450'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_450'] )
train['size_tau_400'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_400'] )
test['size_tau_400'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_400'] )
train['size_tau_300'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_300'] )
test['size_tau_300'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_300'] )
#train['size_tau_150'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_150'] )
#test['size_tau_150'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_150'] )
train['size_tau_200'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_200'] )
test['size_tau_200'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_200'] )

In [ ]:
train['size_tau2'] = np.sqrt( 1/ train['trade_order_count_sum'] )
test['size_tau2'] = np.sqrt( 1/ test['trade_order_count_sum'] )
#train['size_tau2_450'] = np.sqrt( 0.25/ train['trade_order_count_sum'] )
#test['size_tau2_450'] = np.sqrt( 0.25/ test['trade_order_count_sum'] )
train['size_tau2_400'] = np.sqrt( 0.33/ train['trade_order_count_sum'] )
test['size_tau2_400'] = np.sqrt( 0.33/ test['trade_order_count_sum'] )
train['size_tau2_300'] = np.sqrt( 0.5/ train['trade_order_count_sum'] )
test['size_tau2_300'] = np.sqrt( 0.5/ test['trade_order_count_sum'] )
#train['size_tau2_150'] = np.sqrt( 0.75/ train['trade_order_count_sum'] )
#test['size_tau2_150'] = np.sqrt( 0.75/ test['trade_order_count_sum'] )
train['size_tau2_200'] = np.sqrt( 0.66/ train['trade_order_count_sum'] )
test['size_tau2_200'] = np.sqrt( 0.66/ test['trade_order_count_sum'] )

# tauのちょっとした増分（デルタ）
train['size_tau2_d'] = train['size_tau2_400'] - train['size_tau2']
test['size_tau2_d'] = test['size_tau2_400'] - test['size_tau2']

In [ ]:
# 今trainが持っているデータのカラム名を配列にしたもの
# not inなので{"stock_id", "time_id", "target", "row_id"}は含まない
colNames = [col for col in list(train.columns)
            if col not in {"stock_id", "time_id", "target", "row_id"}]
# 何個のカラムがあるのか確認
len(colNames)

## 縦列time_id, 横列stock_idでクロスするセルにそれぞれのtime_id,stock_idの時のtargetが入っているテーブルが欲しい

In [ ]:
# k近傍法
from sklearn.cluster import KMeans
# making agg features

train_p = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
# indexとcolumnsとvaluesとvaluesを指定してテーブルをピボット（再形成）
# 今回は縦がtime_id, 横がstock_id, クロスするセルに入るのがtime_id,stock_idが特定の値の時のtargetの値
train_p = train_p.pivot(index='time_id', columns='stock_id', values='target')
# 確認
display(train_p.head())

# ピボットしたtrain_pの増えかたを銘柄ごとにまとめたもの
# どの銘柄同士で相関が高いかを確認し、のちのクラスター分析に関わってくる
corr = train_p.corr()

# stock_idが一次元配列で格納されたもの
# ids = [0,1,2,3,4,...,111,112]
ids = corr.index


# k-means法, クラスター数7, random_state=0(同じ値)
# ボラティリティの増え方で銘柄に種類があるのではないか？
kmeans = KMeans(n_clusters=7, random_state=0).fit(corr.values)
# クラスター数がラベルになって0~7が格納されている
print(kmeans.labels_)

l = []
for n in range(7):
    l.append ( [ (x-1) for x in ( (ids+1)*(kmeans.labels_ == n)) if x > 0] )
# appendされているのがx-1の値で、xが0以上の時はkmeans.labels_と0~6の間を動くnが等しい時、ids+1を返す。
# appendされているのがx-1なので（ids + 1）-1の値がl（配列）に格納されlはstock_idの一次元配列になる
# (kmeans.labels_ == n)がTrue,Falseで返ってくるが、pythonではTrue==1,Fales==0として扱われる（だから掛け算が出来ていた）
display(l)
    
mat = []
matTest = []

n = 0
for ind in l:
    print(ind)
    newDf = train.loc[train['stock_id'].isin(ind) ]
    newDf = newDf.groupby(['time_id']).agg(np.nanmean) #meanと一緒
    newDf.loc[:,'stock_id'] = str(n)+'c1'
    mat.append ( newDf )
    
    newDf = test.loc[test['stock_id'].isin(ind) ]    
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = str(n)+'c1'
    matTest.append ( newDf )
    
    n+=1
# ここまでfor文。nが0~6まで（割り振られた全クラスター）でtime_idごとにstock_idごとにボラティリティなどなどの平均をとったdataframeを作成
    
mat1 = pd.concat(mat).reset_index()
mat1.drop(columns=['target'],inplace=True)

mat2 = pd.concat(matTest).reset_index()
display(mat1.head())

In [ ]:
mat2 = pd.concat([mat2,mat1.loc[mat1.time_id==5]])
mat1 = mat1.pivot(index='time_id', columns='stock_id')
mat1.columns = ["_".join(x) for x in mat1.columns.ravel()]
mat1.reset_index(inplace=True)

mat2 = mat2.pivot(index='time_id', columns='stock_id')
mat2.columns = ["_".join(x) for x in mat2.columns.ravel()]
mat2.reset_index(inplace=True)
display(mat2.head())

In [ ]:
# カラム名をまとめた配列を用意
nnn = ['time_id',
     'log_return1_realized_volatility_0c1',
     'log_return1_realized_volatility_1c1',     
     'log_return1_realized_volatility_3c1',
     'log_return1_realized_volatility_4c1',     
     'log_return1_realized_volatility_6c1',
     'total_volume_sum_0c1',
     'total_volume_sum_1c1', 
     'total_volume_sum_3c1',
     'total_volume_sum_4c1', 
     'total_volume_sum_6c1',
     'trade_size_sum_0c1',
     'trade_size_sum_1c1', 
     'trade_size_sum_3c1',
     'trade_size_sum_4c1', 
     'trade_size_sum_6c1',
     'trade_order_count_sum_0c1',
     'trade_order_count_sum_1c1',
     'trade_order_count_sum_3c1',
     'trade_order_count_sum_4c1',
     'trade_order_count_sum_6c1',      
     'price_spread_sum_0c1',
     'price_spread_sum_1c1',
     'price_spread_sum_3c1',
     'price_spread_sum_4c1',
     'price_spread_sum_6c1',   
     'bid_spread_sum_0c1',
     'bid_spread_sum_1c1',
     'bid_spread_sum_3c1',
     'bid_spread_sum_4c1',
     'bid_spread_sum_6c1',       
     'ask_spread_sum_0c1',
     'ask_spread_sum_1c1',
     'ask_spread_sum_3c1',
     'ask_spread_sum_4c1',
     'ask_spread_sum_6c1',   
     'volume_imbalance_sum_0c1',
     'volume_imbalance_sum_1c1',
     'volume_imbalance_sum_3c1',
     'volume_imbalance_sum_4c1',
     'volume_imbalance_sum_6c1',       
     'bid_ask_spread_sum_0c1',
     'bid_ask_spread_sum_1c1',
     'bid_ask_spread_sum_3c1',
     'bid_ask_spread_sum_4c1',
     'bid_ask_spread_sum_6c1',
     'size_tau2_0c1',
     'size_tau2_1c1',
     'size_tau2_3c1',
     'size_tau2_4c1',
     'size_tau2_6c1'] 
train = pd.merge(train,mat1[nnn],how='left',on='time_id')
test = pd.merge(test,mat2[nnn],how='left',on='time_id')

In [ ]:
import gc
del mat1,mat2
gc.collect()

In [ ]:
# 交差検証をする時にKfoldを使う
# データをk個に分け，n個を訓練用，k-n個をテスト用として使う．
# 分けられたn個のデータがテスト用として必ず1回使われるようにn回検定する．
from sklearn.model_selection import KFold
# lightgbm
import lightgbm as lgb

seed0=2021
params0 = {
    'objective': 'rmse',
    'boosting_type': 'gbdt',
    'max_depth': -1,
    'max_bin':100,
    'min_data_in_leaf':500,
    'learning_rate': 0.05,
    'subsample': 0.72,
    'subsample_freq': 4,
    'feature_fraction': 0.5,
    'lambda_l1': 0.5,
    'lambda_l2': 1.0,
    'categorical_column':[0],
    'seed':seed0,
    'feature_fraction_seed': seed0,
    'bagging_seed': seed0,
    'drop_seed': seed0,
    'data_random_seed': seed0,
    'n_jobs':-1,
    'verbose': -1}
seed1=42
params1 = {
        'learning_rate': 0.1,        
        'lambda_l1': 2,
        'lambda_l2': 7,
        'num_leaves': 800,
        'min_sum_hessian_in_leaf': 20,
        'feature_fraction': 0.8,
        'feature_fraction_bynode': 0.8,
        'bagging_fraction': 0.9,
        'bagging_freq': 42,
        'min_data_in_leaf': 700,
        'max_depth': 4,
        'categorical_column':[0],
        'seed': seed1,
        'feature_fraction_seed': seed1,
        'bagging_seed': seed1,
        'drop_seed': seed1,
        'data_random_seed': seed1,
        'objective': 'rmse',
        'boosting': 'gbdt',
        'verbosity': -1,
        'n_jobs':-1,
    }
# rmspeで早期停止をするための関数
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

# LightGBMを実装するための関数
def train_and_evaluate_lgb(train, test, params):
    
    # time_id, target, row_id以外をfeaturesに格納
    features = [col for col in train.columns if col not in {"time_id", "target", "row_id"}]
    y = train['target']
    # trainと同じ形で0が入った配列を作成
    oof_predictions = np.zeros(train.shape[0])
    # testと同じ形で0が入った配列を作成
    test_predictions = np.zeros(test.shape[0])
    # kfold
    kfold = KFold(n_splits = 5, random_state = 2021, shuffle = True)
    # 各foldで繰り返す
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(train)):
        print(f'Training fold {fold + 1}')
        x_train, x_val = train.iloc[trn_ind], train.iloc[val_ind]
        y_train, y_val = y.iloc[trn_ind], y.iloc[val_ind]
        # rmspe
        train_weights = 1 / np.square(y_train)
        val_weights = 1 / np.square(y_val)
        train_dataset = lgb.Dataset(x_train[features], y_train, weight = train_weights)
        val_dataset = lgb.Dataset(x_val[features], y_val, weight = val_weights)
        model = lgb.train(params = params,
                          num_boost_round=1000,
                          train_set = train_dataset, 
                          valid_sets = [train_dataset, val_dataset], 
                          verbose_eval = 250,
                          early_stopping_rounds=50,
                          feval = feval_rmspe)
        # 各foldに取得した予測値を追加する
        oof_predictions[val_ind] = model.predict(x_val[features])
        # 作成したモデルでtestデータを予測する
        test_predictions += model.predict(test[features]) / 5
    rmspe_score = rmspe(y, oof_predictions)
    print(f'Our out of folds RMSPE is {rmspe_score}')
    lgb.plot_importance(model,max_num_features=20)
    # testの予測値を返り値に設定
    return test_predictions

# 実際に実行してみる
predictions_lgb= train_and_evaluate_lgb(train, test,params0)
test['target'] = predictions_lgb
test[['row_id', 'target']].to_csv('submission.csv',index = False)

圧倒的stock_id

In [ ]:
train.shape[1]

下記を実行するにあたって最初
```
from keras import backend as K
```
を行なった際にNo moduleとなってしまったのでpip listで調べたところversionが2.6.0だったのでversionを変更すると回るようになったので注意
```
!pip uninstall -y keras
!pip install keras==2.3.1
```

In [ ]:
!pip install keras==2.3.1

In [ ]:
from numpy.random import seed
seed(42)

# ディープラーニング向けライブラリtensorflow
import tensorflow as tf
tf.random.set_seed(42)
from tensorflow import keras
import numpy as np
from keras import backend as K

# rmspeを計算
def root_mean_squared_per_error(y_true, y_pred):
         return K.sqrt(K.mean(K.square( (y_true - y_pred)/ y_true )))

# kerasでコールバックを作成する
# こちらはEarlyStopping
# 学習ループに収束判定を付与することができ、監視する値を設定して、それが収束したら自動的にループを抜ける処理になる
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=20, verbose=0,
    mode='min',restore_best_weights=True)
# こちらはReduceLROnPlateau
# 評価値の改善が止まった時に学習率を減らす
plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.2, patience=7, verbose=0,
    mode='min')

In [ ]:
# 再度trainデータの読み込みとテーブルの再形成
out_train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
out_train = out_train.pivot(index='time_id', columns='stock_id', values='target')

# 欠損値を平均値で補完
out_train = out_train.fillna(out_train.mean())
out_train.head()

# 最初の実行後に読み取ったデータだけを追加するコード

# knnに基づいてdataを分ける
nfolds = 5 # foldの数
index = []
totDist = []
values = []
# out_trainの値で行列を生成
mat = out_train.values

# 正規化
scaler = MinMaxScaler(feature_range=(-1, 1))
mat = scaler.fit_transform(mat)

nind = int(mat.shape[0]/nfolds) # foldごとのデータ数

# 最後の列にindexを追加
mat = np.c_[mat,np.arange(mat.shape[0])]

# np.random.choice()でサイコロを作成する
lineNumber = np.random.choice(np.array(mat.shape[0]), size=nfolds, replace=False)

lineNumber = np.sort(lineNumber)[::-1]

# totDistにmatの行数 - 5（foldの数）分の0配列を挿入
for n in range(nfolds):
    totDist.append(np.zeros(mat.shape[0]-nfolds))

# valuesにはindexを保存
for n in range(nfolds):
    values.append([lineNumber[n]])

display(mat)
display(lineNumber)
display(totDist)
display(values)

In [ ]:
s=[]
for n in range(nfolds): #range(5)
    # sにmatの(lineNumberのn行目)の配列の要素をappend
    s.append(mat[lineNumber[n],:])
    # sに入った要素はmatからは削除
    mat = np.delete(mat, obj=lineNumber[n], axis=0)

# nindはfoldごとのデータ数（int(mat.shape[0]/nfolds)）
for n in range(nind-1):    
    # 最小値0, 最大値1の乱数をnfolds個生成（今は5個）
    luck = np.random.uniform(0,1,nfolds)
    
    for cycle in range(nfolds): #range(5)
        # np.matlib.repmatでs[cycle]をmatの行数分の行*1列の行列に変換
        s[cycle] = np.matlib.repmat(s[cycle], mat.shape[0], 1)
        # matの全行前列とsの全行前列の行列式を計算した値を2倍したものをsumDistに代入
        sumDist = np.sum( (mat[:,:-1] - s[cycle][:,:-1])**2 , axis=1)
        # toDist[cycle] = toDist[cycle] + sumDist
        totDist[cycle] += sumDist
        
        f = totDist[cycle]/np.sum(totDist[cycle]) # totdistを正規化
        j = 0
        kn = 0
        for val in f:
            j += val
            if (j > luck[cycle]):
                break
            kn +=1
        lineNumber[cycle] = kn
        
        # 上で値が追加されたレコードはtoDistから削除
        for n_iter in range(nfolds):
            
            totDist[n_iter] = np.delete(totDist[n_iter],obj=lineNumber[cycle], axis=0)
            j= 0
        
        s[cycle] = mat[lineNumber[cycle],:]
        values[cycle].append(int(mat[lineNumber[cycle],-1]))
        mat = np.delete(mat, obj=lineNumber[cycle], axis=0)

for n_mod in range(nfolds):
    values[n_mod] = out_train.index[values[n_mod]]

In [ ]:
# 計算上取得されてしまったtrainとtestの正負無限をnanに置換
train.replace([np.inf, -np.inf], np.nan,inplace=True)
test.replace([np.inf, -np.inf], np.nan,inplace=True)
qt_train = []
train_nn=train[colNames].copy()
test_nn=test[colNames].copy()
for col in colNames:
    qt = QuantileTransformer(random_state=21,n_quantiles=2000, output_distribution='normal')
    train_nn[col] = qt.fit_transform(train_nn[[col]])
    test_nn[col] = qt.transform(test_nn[[col]])    
    qt_train.append(qt)

In [ ]:
train_nn[['stock_id','time_id','target']]=train[['stock_id','time_id','target']]
test_nn[['stock_id','time_id']]=test[['stock_id','time_id']]

In [ ]:
# クラスタリング
from sklearn.cluster import KMeans
train_p = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
train_p = train_p.pivot(index='time_id', columns='stock_id', values='target')

corr = train_p.corr()

ids = corr.index

kmeans = KMeans(n_clusters=7, random_state=0).fit(corr.values)
print(kmeans.labels_)

l = []
for n in range(7):
    l.append ( [ (x-1) for x in ( (ids+1)*(kmeans.labels_ == n)) if x > 0] )
    

mat = []
matTest = []

n = 0
for ind in l:
    print(ind)
    newDf = train_nn.loc[train_nn['stock_id'].isin(ind) ]
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = str(n)+'c1'
    mat.append ( newDf )
    
    newDf = test_nn.loc[test_nn['stock_id'].isin(ind) ]    
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = str(n)+'c1'
    matTest.append ( newDf )
    
    n+=1
    
mat1 = pd.concat(mat).reset_index()
mat1.drop(columns=['target'],inplace=True)

mat2 = pd.concat(matTest).reset_index()
mat2 = pd.concat([mat2,mat1.loc[mat1.time_id==5]])

In [ ]:
nnn = ['time_id',
     'log_return1_realized_volatility_0c1',
     'log_return1_realized_volatility_1c1',     
     'log_return1_realized_volatility_3c1',
     'log_return1_realized_volatility_4c1',     
     'log_return1_realized_volatility_6c1',
     'total_volume_sum_0c1',
     'total_volume_sum_1c1', 
     'total_volume_sum_3c1',
     'total_volume_sum_4c1', 
     'total_volume_sum_6c1',
     'trade_size_sum_0c1',
     'trade_size_sum_1c1', 
     'trade_size_sum_3c1',
     'trade_size_sum_4c1', 
     'trade_size_sum_6c1',
     'trade_order_count_sum_0c1',
     'trade_order_count_sum_1c1',
     'trade_order_count_sum_3c1',
     'trade_order_count_sum_4c1',
     'trade_order_count_sum_6c1',      
     'price_spread_sum_0c1',
     'price_spread_sum_1c1',
     'price_spread_sum_3c1',
     'price_spread_sum_4c1',
     'price_spread_sum_6c1',   
     'bid_spread_sum_0c1',
     'bid_spread_sum_1c1',
     'bid_spread_sum_3c1',
     'bid_spread_sum_4c1',
     'bid_spread_sum_6c1',       
     'ask_spread_sum_0c1',
     'ask_spread_sum_1c1',
     'ask_spread_sum_3c1',
     'ask_spread_sum_4c1',
     'ask_spread_sum_6c1',   
     'volume_imbalance_sum_0c1',
     'volume_imbalance_sum_1c1',
     'volume_imbalance_sum_3c1',
     'volume_imbalance_sum_4c1',
     'volume_imbalance_sum_6c1',       
     'bid_ask_spread_sum_0c1',
     'bid_ask_spread_sum_1c1',
     'bid_ask_spread_sum_3c1',
     'bid_ask_spread_sum_4c1',
     'bid_ask_spread_sum_6c1',
     'size_tau2_0c1',
     'size_tau2_1c1',
     'size_tau2_3c1',
     'size_tau2_4c1',
     'size_tau2_6c1']

In [ ]:
# 先ほどmat2はtime_idが5の時に限定していたが今回は全データ
mat1 = mat1.pivot(index='time_id', columns='stock_id')
mat1.columns = ["_".join(x) for x in mat1.columns.ravel()]
mat1.reset_index(inplace=True)

mat2 = mat2.pivot(index='time_id', columns='stock_id')
mat2.columns = ["_".join(x) for x in mat2.columns.ravel()]
mat2.reset_index(inplace=True)
display(mat2)

In [ ]:
import gc
train_nn = pd.merge(train_nn,mat1[nnn],how='left',on='time_id')
test_nn = pd.merge(test_nn,mat2[nnn],how='left',on='time_id')
del mat1,mat2
del train,test
gc.collect()

## ニューラルネットワーク

In [ ]:
# https://bignerdranch.com/blog/implementing-swish-activation-function-in-keras/
# sigmoid関数
from keras.backend import sigmoid
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))

from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
get_custom_objects().update({'swish': Activation(swish)})

hidden_units = (128,64,32)
stock_embedding_size = 24

cat_data = train_nn['stock_id']

def base_model():
    # ニューラルネットワーク入力層（shapeは入力の次元）
    stock_id_input = keras.Input(shape=(1,), name='stock_id')
    num_input = keras.Input(shape=(244,), name='num_data')

    #embedding（埋め込み）, flatenning（平坦化, concatenating（連結）
    stock_embedded = keras.layers.Embedding(max(cat_data)+1, stock_embedding_size, 
                                           input_length=1, name='stock_embedding')(stock_id_input)
    stock_flattened = keras.layers.Flatten()(stock_embedded)
    out = keras.layers.Concatenate()([stock_flattened, num_input])
    
    # 隠れ層
    for n_hidden in hidden_units:

        out = keras.layers.Dense(n_hidden, activation='swish')(out)
        

    # out = keras.layers.Concatenate()([out, num_input])
    # 出力層
    out = keras.layers.Dense(1, activation='linear', name='prediction')(out)
    
    model = keras.Model(
    inputs = [stock_id_input, num_input],
    outputs = out,
    )
    
    return model

In [ ]:
# もう一度関数を設定している
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

In [ ]:
target_name='target'
scores_folds = {}
model_name = 'NN'
pred_name = 'pred_{}'.format(model_name)

n_folds = 5
kf = model_selection.KFold(n_splits=n_folds, shuffle=True, random_state=2020)
scores_folds[model_name] = []
counter = 1

features_to_consider = list(train_nn)

features_to_consider.remove('time_id')
features_to_consider.remove('target')
try:
    features_to_consider.remove('pred_NN')
except:
    pass


train_nn[features_to_consider] = train_nn[features_to_consider].fillna(train_nn[features_to_consider].mean())
test_nn[features_to_consider] = test_nn[features_to_consider].fillna(train_nn[features_to_consider].mean())

train_nn[pred_name] = 0
test_nn[target_name] = 0
test_predictions_nn = np.zeros(test_nn.shape[0])

for n_count in range(n_folds):
    print('CV {}/{}'.format(counter, n_folds))
    
    indexes = np.arange(nfolds).astype(int)    
    indexes = np.delete(indexes,obj=n_count, axis=0) 
    
    indexes = np.r_[values[indexes[0]],values[indexes[1]],values[indexes[2]],values[indexes[3]]]
    
    X_train = train_nn.loc[train_nn.time_id.isin(indexes), features_to_consider]
    y_train = train_nn.loc[train_nn.time_id.isin(indexes), target_name]
    X_test = train_nn.loc[train_nn.time_id.isin(values[n_count]), features_to_consider]
    y_test = train_nn.loc[train_nn.time_id.isin(values[n_count]), target_name]
    
    #############################################################################################
    # NN
    #############################################################################################
    
    model = base_model()
    
    model.compile(
        keras.optimizers.Adam(learning_rate=0.006),
        loss=root_mean_squared_per_error
    )
    
    try:
        features_to_consider.remove('stock_id')
    except:
        pass
    
    num_data = X_train[features_to_consider]
    
    scaler = MinMaxScaler(feature_range=(-1, 1))         
    num_data = scaler.fit_transform(num_data.values)    
    
    cat_data = X_train['stock_id']    
    target =  y_train
    
    num_data_test = X_test[features_to_consider]
    num_data_test = scaler.transform(num_data_test.values)
    cat_data_test = X_test['stock_id']

    model.fit([cat_data, num_data], 
              target,               
              batch_size=2048,
              epochs=1000,
              validation_data=([cat_data_test, num_data_test], y_test),
              callbacks=[es, plateau],
              validation_batch_size=len(y_test),
              shuffle=True,
             verbose = 1)

    preds = model.predict([cat_data_test, num_data_test]).reshape(1,-1)[0]
    
    score = round(rmspe(y_true = y_test, y_pred = preds),5)
    print('Fold {} {}: {}'.format(counter, model_name, score))
    scores_folds[model_name].append(score)
    
    tt =scaler.transform(test_nn[features_to_consider].values)
    #test_nn[target_name] += model.predict([test_nn['stock_id'], tt]).reshape(1,-1)[0].clip(0,1e10)
    test_predictions_nn += model.predict([test_nn['stock_id'], tt]).reshape(1,-1)[0].clip(0,1e10)/n_folds
    #test[target_name] += model.predict([test['stock_id'], test[features_to_consider]]).reshape(1,-1)[0].clip(0,1e10)
       
    counter += 1
    features_to_consider.append('stock_id')

In [ ]:
test_nn["row_id"] = test_nn["stock_id"].astype(str) + "-" + test_nn["time_id"].astype(str) 
test_nn[target_name] = (test_predictions_nn+predictions_lgb)/2

score = round(rmspe(y_true = train_nn[target_name].values, y_pred = train_nn[pred_name].values),5)
print('RMSPE {}: {} - Folds: {}'.format(model_name, score, scores_folds[model_name]))

display(test_nn[['row_id', target_name]].head(3))
test_nn[['row_id', target_name]].to_csv('submission.csv',index = False)
#test[['row_id', target_name]].to_csv('submission.csv',index = False)
#kmeans N=5 [0.2101, 0.21399, 0.20923, 0.21398, 0.21175]

一通り流れを追って実行してみたが、1週目ではイマイチわからない箇所が多かった。

他のnotebookも拝見させていただきながら周回してどのような特徴量を生成しているのかを中心に復習する。